In [ ]:
red_light_path = "/Users/shingkai/code/personal-projects/mk8/training_data/race_start/frame_000394.jpg"
green_light_path = "/Users/shingkai/code/personal-projects/mk8/training_data/race_start/frame_000395.jpg"

In [ ]:
import matplotlib.pyplot as plt
import cv2

image = cv2.imread(green_light_path)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [ ]:
# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Set up matplotlib style for better notebook display
plt.rcParams['figure.figsize'] = (12, 6)

def plot_image_with_histogram(image, title="Image Analysis"):
    """
    Display both the image and its histogram side by side in a notebook.
    
    Parameters:
    image (numpy.ndarray): Input CV2 image
    title (str): Title for the plots
    """
    # Create subplot layout
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot the image
    if len(image.shape) == 3:
        # Convert BGR to RGB for display
        display_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        display_img = image
    ax1.imshow(display_img)
    ax1.set_title(f"{title}")
    ax1.axis('off')
    
    # Plot histogram
    if len(image.shape) == 2 or image.shape[2] == 1:
        # Grayscale histogram
        hist = cv2.calcHist([image], [0], None, [256], [0, 256])
        ax2.plot(hist, color='black', label='Intensity')
        ax2.set_xlim([0, 256])
        ax2.set_title("Grayscale Histogram")
    else:
        # Color histogram
        colors = ('r', 'g', 'b')
        for i, col in enumerate(colors):
            hist = cv2.calcHist([image], [i], None, [256], [0, 256])
            ax2.plot(hist, color=col, label=col.upper())
        ax2.set_xlim([0, 256])
        ax2.set_title("Color Histogram")
    
    ax2.set_xlabel('Pixel Value')
    ax2.set_ylabel('Frequency')
    ax2.legend()
    ax2.grid(True, alpha=0.2)
    
    plt.tight_layout()
    plt.show()

def analyze_image_histograms(image_path):
    """
    Comprehensive histogram analysis of an image.
    Shows original, grayscale, and equalized versions with their histograms.
    
    Parameters:
    image_path (str): Path to the input image
    """
    # Load image
    original = cv2.imread(image_path)
    if original is None:
        raise ValueError(f"Could not load image from {image_path}")
    
    # Original image analysis
    plot_image_with_histogram(original, "Original Image")



In [ ]:
analyze_image_histograms(red_light_path)
analyze_image_histograms(green_light_path)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def compare_roi_histograms(images, regions, titles=None):
    """
    Compare histograms from regions of one or two images.
    
    Parameters:
    images: Either a single image or tuple/list of two images
    regions: List of tuples [(x1, y1, x2, y2), ...] for ROI coordinates
    titles: Optional list of titles for each region
    """
    # Handle single or multiple images
    if isinstance(images, (list, tuple)):
        image1, image2 = images
        multi_image = True
    else:
        image1 = image2 = images
        multi_image = False
    
    # Validate inputs
    if not isinstance(regions, list):
        raise ValueError("regions must be a list of coordinate tuples")
    
    if titles is None:
        titles = [f"Region {i+1}" for i in range(len(regions))]
    
    # Validate image dimensions
    h1, w1 = image1.shape[:2]
    h2, w2 = image2.shape[:2]
    
    # Set up subplots - one column for each region
    fig, axes = plt.subplots(2, len(regions), figsize=(6*len(regions), 10))
    
    # Ensure axes is 2D even with single region
    if len(regions) == 1:
        axes = axes.reshape(-1, 1)
    
    for i, ((x1, y1, x2, y2), title) in enumerate(zip(regions, titles)):
        # Determine which image to use for this region
        img = image2 if (multi_image and i >= len(regions)//2) else image1
        h, w = img.shape[:2]
        
        # Validate and adjust coordinates
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)
        
        if x1 >= x2 or y1 >= y2:
            raise ValueError(f"Invalid coordinates for region {i+1}: ({x1},{y1}) to ({x2},{y2})")
        
        # Extract ROI
        roi = img[y1:y2, x1:x2]
        
        # Show ROI
        if len(img.shape) == 3:
            display_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        else:
            display_roi = roi
            
        axes[0, i].imshow(display_roi)
        axes[0, i].set_title(f"{title}\n({x1}, {y1}) to ({x2}, {y2})")
        axes[0, i].axis('off')
        
        # Plot histogram
        if len(img.shape) == 2 or img.shape[2] == 1:
            # Grayscale histogram
            hist = cv2.calcHist([roi], [0], None, [256], [0, 256])
            axes[1, i].plot(hist, color='black', label='Intensity')
            axes[1, i].set_title("Grayscale Histogram")
        else:
            # Color histogram
            colors = ('b', 'g', 'r')
            for j, col in enumerate(colors):
                hist = cv2.calcHist([roi], [j], None, [256], [0, 256])
                axes[1, i].plot(hist, color=col, label=col.upper())
            axes[1, i].set_title("Color Histogram")
        
        axes[1, i].set_xlim([250, 256])
        axes[1, i].set_xlabel('Pixel Value')
        axes[1, i].set_ylabel('Frequency')
        axes[1, i].legend()
        axes[1, i].grid(True, alpha=0.2)
        
        # Add ROI statistics
        stats = f"Mean: {np.mean(roi):.1f}\nStd: {np.std(roi):.1f}"
        axes[1, i].text(0.02, 0.98, stats, 
                       transform=axes[1, i].transAxes,
                       verticalalignment='top',
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()


In [ ]:
image1 = cv2.imread(red_light_path)
image2 = cv2.imread(green_light_path)
regions = [
    (170, 220, 350, 300),
    (170, 220, 350, 300)
]
# regions = [
#     (0, 0, 1920, 1080),
#     (0, 0, 1920, 1080)
# ]
titles = ["Red Light Region", "Green Light Region"]
compare_roi_histograms([image1, image2], regions, titles)



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg

def process_video(video_path, roi):
    """
    Process video file and display ROI with histogram analysis
    
    Args:
        video_path (str): Path to video file
        roi (tuple): Region of interest as (x, y, width, height)
    """
    # Open video file
    cap = cv2.VideoCapture(video_path)
    
    # Create windows
    cv2.namedWindow('ROI', cv2.WINDOW_NORMAL)
    cv2.namedWindow('Histogram', cv2.WINDOW_NORMAL)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Extract ROI
        x, y, w, h = roi
        roi_frame = frame[y:y+h, x:x+w]
        
        # Count green pixels over 250
        green_channel = roi_frame[:,:,1]
        green_pixels = np.sum(green_channel > 250)
        
        # Create histogram
        fig = plt.figure(figsize=(8, 4))
        plt.hist(green_channel.ravel(), bins=6, range=[250,256], color='green')
        plt.title(f'Green Channel Histogram\nPixels > 250: {green_pixels}')
        plt.xlabel('Pixel Value')
        plt.ylabel('Count')
        
        # Convert matplotlib figure to OpenCV image
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        hist_image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
        hist_image = hist_image.reshape(canvas.get_width_height()[::-1] + (3,))
        hist_image = cv2.cvtColor(hist_image, cv2.COLOR_RGB2BGR)
        plt.close()
        
        # Display ROI and histogram
        cv2.imshow('ROI', roi_frame)
        cv2.imshow('Histogram', hist_image)
        
        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    

In [ ]:
video_path = "../../race_videos/clip_1080_start.mp4"
roi = (120, 160, 200, 100)  # x, y, width, height
process_video(video_path, roi)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
from collections import deque

def process_video(video_path, roi, history_length=100):
    """
    Process video file and display ROI with time series of red/green pixel counts
    
    Args:
        video_path (str): Path to video file
        roi (tuple): Region of interest as (x, y, width, height)
        history_length (int): Number of frames to show in time series
    """
    cap = cv2.VideoCapture(video_path)
    
    # Create windows
    cv2.namedWindow('ROI', cv2.WINDOW_NORMAL)
    cv2.namedWindow('Time Series', cv2.WINDOW_NORMAL)
    
    # Initialize deques for storing historical values
    red_counts = deque(maxlen=history_length)
    green_counts = deque(maxlen=history_length)
    
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Extract ROI
        if roi:
            x, y, w, h = roi
            roi_frame = frame[y:y+h, x:x+w]
        else:
            roi_frame = frame
        
        # Convert to HSV
        hsv = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2HSV)
        
        # Split into channels
        b, g, r = cv2.split(roi_frame)
        
        # Count red pixels (high red, low blue/green)
        red_mask = (r > 230) & (b < 150) & (b > 50) & (g < 200) & (g > 90)
        red_pixel_count = np.sum(red_mask)
        
        # Count green pixels (high green, low red/blue)
        green_mask = (g > 250) & (r < 250) & (r > 150) & (b < 250) & (b > 190)
        green_pixel_count = np.sum(green_mask)
        
        # Add counts to history
        red_counts.append(red_pixel_count)
        green_counts.append(green_pixel_count)
        
        # Create time series plot
        fig = plt.figure(figsize=(10, 4))
        plt.plot(list(red_counts), 'r-', label='Red Pixels')
        plt.plot(list(green_counts), 'g-', label='Green Pixels')
        plt.legend()
        plt.title('Pixel Count Over Time')
        plt.xlabel('Frames')
        plt.ylabel('Pixel Count')
        plt.grid(True)
        
        # Keep y-axis consistent for better visualization
        plt.ylim(0, max(max(red_counts), max(green_counts)) * 1.1)
        
        # Convert matplotlib figure to OpenCV image
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        plot_image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
        plot_image = plot_image.reshape(canvas.get_width_height()[::-1] + (3,))
        plot_image = cv2.cvtColor(plot_image, cv2.COLOR_RGB2BGR)
        plt.close()
        
        # Visualize detected pixels in ROI
        display_frame = roi_frame.copy()
        display_frame[red_mask] = [0, 0, 255]   # Highlight red pixels
        display_frame[green_mask] = [0, 255, 0]  # Highlight green pixels
        
        # Display ROI and time series
        cv2.imshow('ROI', display_frame)
        cv2.imshow('Time Series', plot_image)
        
        frame_count += 1
        
        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
video_path = "../../race_videos/clip_1080_start.mp4"
roi = (100, 160, 250, 150)  # x, y, width, height
roi = (0, 0, 1920, 1080)  # x, y, width, height
process_video(video_path, None)

In [ ]:
process_video("../../race_videos/donut_plains.mp4", (100, 100, 200, 100))